# Objectives
The purpose of this section is to implement a dictionary attack over a WPA2-Personal wifi.To answer most of the following questions, we used the Python cryptography library.

# Preliminaries
File tradio.pcapng is a capture test that contains a WPA2-Personal handshake and some ping packets after handshake for the following wifi session:

*   SSID: Wifi Test
*   Password: Wifi Test Password

You can identify the 4 handshake messages by applying filter eapol in wireshark. Also, you can
decrypt traffic (see: https://wiki.wireshark.org/HowToDecrypt802.11).

In [1]:
def validate_solution(str1, str2_start, str2_end):
    # Compare the first 4 characters and the last 4 characters
    if str1[:4] == str2_start and str1[-4:] == str2_end:
        return True
    else:
        return False

# Question 1
For WPA2 personal, PMK is PSK. Prove that PMK is: a22c...b603

The first thing we have to do is to derive the PMK or also called PSK from:


*   SSID
*   Password




In [2]:
from cryptography.hazmat.primitives.kdf.pbkdf2  import PBKDF2HMAC
from cryptography.hazmat.primitives import hashes

ssid = "Wifi_Test"  # SSID (Wi-Fi network name)
password = "Wifi_Test_Password"     # Wi-Fi password

# Generate salt based on the SSID
salt = ssid.encode()

# Define parameters for PBKDF2 key derivation function
key_length = 32    # Length of the derived key (PMK) in bytes
iterations = 4096  # Number of iterations

# Derive PMK using PBKDF2HMAC key derivation function
kdf = PBKDF2HMAC(
        algorithm=hashes.SHA1(),
        length=key_length,
        salt=salt,
        iterations=iterations,
)
pmk = kdf.derive(password.encode())

# Convert PMK bytes to hexadecimal representation
pmk_hex = pmk.hex()
print("Derived PMK:", pmk_hex)
print(validate_solution(pmk_hex,"a22c","b603"))

Derived PMK: a22c4740d1843aca77252e5a6923af13773be449bb443edca5c6296ca151b603
True


# Question 2.
 Parse the capture file 'tradio.pcapng' with tshark and dump the required handshake packets in hexadecimal. From the first handshake message, prove that nonce is b6ea...d7d3b. From the second handshake message, prove that nonce is e6d8...b61c.


Instead of using tshark, we decided to use pyshark to parse the captured file and also dump the required handshake packets.


In [3]:
import pyshark
import nest_asyncio

nest_asyncio.apply()

handshake_messages = pyshark.FileCapture(input_file="tradio.pcapng", display_filter='eapol')

# Set use_json and include_raw to True
capture = pyshark.FileCapture(input_file="tradio.pcapng", display_filter='eapol', use_json=True, include_raw=True)

packet_1_raw = capture[0].get_raw_packet()
packet_2_raw = capture[1].get_raw_packet()

print(packet_1_raw[77:109].hex())
print(validate_solution(packet_1_raw[77:109].hex(),"b6ea","7d3b"))
print(packet_2_raw[77:109].hex())
print(validate_solution(packet_2_raw[77:109].hex(),"e6d8","b61c"))

b6ea13c99b3a6aeeb973b622d5671d3f4a143effe67660c0bc01b774604d7d3b
True
e6d818b941847156d83eab024dc81f8ce372ab4a625bf0f80c383c536578b61c
True


# Question 3.
 Prove that PTK is 0ffc...8aad.

First of all we need to extract de MAC address and nonce from the capture.

In [4]:
MAC_add1 = packet_1_raw[30:36]
MAC_add2 = packet_1_raw[36:42]
nonce_1 = packet_1_raw[77:109]
nonce_2 = packet_2_raw[77:109]

Once we have those, we can calculate the ptk, using the provided functions from the diapos (100).

In [5]:
from cryptography.hazmat.primitives import hmac

def HMAC(key, algo, input):
    h = hmac.HMAC(key, algo)
    h.update(input)
    return h.finalize()

def SHA1_HMAC(key, input, length):
    r = bytes()
    for i in range(1 + length // 20):
        r+= HMAC(key, hashes.SHA1(), input + chr(i).encode())
    return r[:length]

# ordered the fields (less first)
mac_1, mac_2 = min(MAC_add1, MAC_add2) , max(MAC_add1, MAC_add2)
nonce_1, nonce_2 = min(nonce_1, nonce_2), max(nonce_1, nonce_2)
data = 'Pairwise key expansion'.encode() + chr(0x00).encode() + mac_1 + mac_2 + nonce_1 + nonce_2
PTK = SHA1_HMAC(pmk, data, 48)

print("PTK:" + bytes.hex(PTK))
print(validate_solution(bytes.hex(PTK),"0ffc","8aad"))

PTK:0ffc1910a2ff9e32a260c3e10427ff980749105d9cfdf27cf232798126a306ef5d75e703a06de9d51b492dfdb9138aad
True


# Question 4.
Prove the authenticity of handshake messages 2, 3 and 4.

First of all we have to extract the KCK, KEK and TK keys from the PTK.

In [6]:
KCK = PTK[:16]
KEK = PTK[16:32]
TK = PTK[32:48]

Once we have those keys, we can compute the MIC, which then we can verify if it's equal to the MIC of the packet.

In [7]:
def compute_mic(KCK, mess):
    data = mess[:81]
    for _ in range(16):
        data+=chr(0x00).encode()
    data += mess[81+16:]
    return HMAC(KCK, hashes.SHA1(), data)[:16]

To verify if it's the same, we have to search in the packets for the MIC and compare it with the results.

In [8]:
# PROVAR DE AGAFAR EL FITXER EN RAW I EAPOL AL MATEIX FILTRE.
capture_file = './tradio.pcapng'
filter_expr = 'eapol'

# Set use_json and include_raw to True
capture = pyshark.FileCapture(capture_file, display_filter=filter_expr, use_json =True, include_raw=True)
raw_handshake_messages = []

for packet in capture:
    raw_handshake_messages.append(packet)


handshake_message_2_mess = raw_handshake_messages[1].get_raw_packet()
handshake_message_3_mess= raw_handshake_messages[2].get_raw_packet()
handshake_message_4_mess = raw_handshake_messages[3].get_raw_packet()


handshake_message_2_mic = compute_mic(KCK, handshake_message_2_mess[60:])
handshake_message_3_mic = compute_mic(KCK, handshake_message_3_mess[60:])
handshake_message_4_mic = compute_mic(KCK, handshake_message_4_mess[60:])

print(bytes.hex(handshake_message_2_mic))
print(validate_solution(bytes.hex(handshake_message_2_mic),"27c4","582c"))
print(bytes.hex(handshake_message_3_mic))
print(validate_solution(bytes.hex(handshake_message_3_mic),"35db","02ed"))
print(bytes.hex(handshake_message_4_mic))
print(validate_solution(bytes.hex(handshake_message_4_mic),"a29e","d947"))


27c403fe1ebc7060c97748972dc3582c
True
35db4be1b17eab872f11066ef1f402ed
True
a29e3a96166c27cad54d30eb8644d947
True


# 1.3 Decrypt unicast data
Now is time to decrypt a data packet. Note that it is a QoS Data Frame. You can use packet
number 517 which is a ping request. Encryption is done using AES128-CCMP.

# Question 5.
Prove that nonce is: 002269a9e50b3500000000000b
First of all we have to find and parse the 517 packet.

In [9]:
capture_file = './tradio.pcapng'
filter_expr = 'frame.number==517'

# Set use_json and include_raw to True
unicast_capture = pyshark.FileCapture(capture_file, display_filter=filter_expr, use_json =True, include_raw=True)

# We get the bytes from the packet
raw_pckg517 = unicast_capture[0].get_raw_packet()

# We need to calculate the length to know where starts the information
length = int.from_bytes(raw_pckg517[2:4],'little')

unicast_raw_pckg517 = raw_pckg517[length:]


Once we have the packet, to prove the nonce we have to compute it, according to the diapo (101) we have to consider that the frame has a QoS field, so we compute it like this.
nonce = QoSControl[0:1] + addr_2 + CCMPpar[0:2] + CCMpar[4:8]


In [10]:
# We get from the raw packet all the needed parts to derive the nonce.
QoSControl = unicast_raw_pckg517[24:26]
CCMPpar = (unicast_raw_pckg517[26:29] + chr(0x00).encode() + unicast_raw_pckg517[30:34])[::-1]
add_2 = unicast_raw_pckg517[10:16]

# We use the above function to derive it.
nonce = QoSControl[0:1] + add_2 + CCMPpar[0:2] + CCMPpar[4:8]
print(nonce.hex())
print(validate_solution(nonce.hex(),"0022","000b"))


002269a9e50b3500000000000b
True


# Question 6.
Prove that AAD is: 884184aa9cfd08202269a9e50b3584aa9cfd081f00000000
According to the diapo (101) we have to consider that the frame has a QoS field, so we compute it like this.
AAD = FrameControl (2 bytes) || Address 1 (6 bytes) || Address 2 || Address 3 ||0x0000 || QoSControl[0:2]


In [11]:
# We get from the raw packet all the needed parts to derive the AAD.
fc = unicast_raw_pckg517[0:2]
add_1 = unicast_raw_pckg517[4:10]
add_3 = unicast_raw_pckg517[16:22]

# We use the above funtion to derive it.
AAD = fc[0:2] + add_1[0:6] + add_2 + add_3 + chr(0x0000).encode() + chr(0x00).encode() + QoSControl[0:2]
print(AAD.hex())
print(validate_solution(AAD.hex(),"8841","0000"))

884184aa9cfd08202269a9e50b3584aa9cfd081f00000000
True


# Question 7.
Prove that decrypted plaintext is: aaaa...3637
According to the diapo (101) we have to consider that it's a unicast packet, so we compute it like this.
aesccm = AESCCM(key) and aesccm.decrypt(nonce, ct, aad), where the key is TK.


In [12]:
from cryptography.hazmat.primitives.ciphers.aead import AESCCM

data_to_decrypt = unicast_raw_pckg517[34:]
unicast_nonce = nonce #We take the previous derived nonce. 

aesccm = AESCCM(TK, tag_length=8)
decrypted_data= aesccm.decrypt(unicast_nonce, data_to_decrypt, AAD)

print(decrypted_data.hex())
print(decrypted_data.hex() == 'aaaa03000000080045000054ba9240004001fcb7c0a8010dc0a801010800d14c00020001016821640000000036110f0000000000101112131415161718191a1b1c1d1e1f202122232425262728292a2b2c2d2e2f3031323334353637')

aaaa03000000080045000054ba9240004001fcb7c0a8010dc0a801010800d14c00020001016821640000000036110f0000000000101112131415161718191a1b1c1d1e1f202122232425262728292a2b2c2d2e2f3031323334353637
True


# 1.4 Decrypt multicast/broadcast data
In the same capture, there is also multicast/broadcast traffic, such as packet 527, that can not
be decrypted with the same key. Note that it is not a QoS Data Frame. First we have to parse the packet 527, like we did before with the 517.

In [13]:
capture_file = './tradio.pcapng'
filter_expr = 'frame.number==527'

# Set use_json and include_raw to True
multicast_capture = pyshark.FileCapture(capture_file, display_filter=filter_expr, use_json =True, include_raw=True)

# We get the bytes from the multicast packet.
raw_pckg527 = multicast_capture[0].get_raw_packet()

# We need to calculate the length to know where starts the information
length = int.from_bytes(raw_pckg527[2:4],'little')
multicast_raw_pckg527 = raw_pckg527[length:]

## Question 8.
Obtain GTK and prove that is: f3b1...4ec7. 

Once we have the packet, in order to obtain the GTK, we need to unwrap it from the 3rd handshake message, like this.
GTK=AES UNWRAP(key=KEK, in=data)[30:46]

In [14]:
from cryptography.hazmat.primitives.keywrap import aes_key_unwrap

wrapped_key = raw_handshake_messages[2].get_raw_packet()
GTK = aes_key_unwrap(KEK,wrapped_key[159:])[30:46]
print(GTK.hex())
print(validate_solution(GTK.hex(),"f3b1","4ec7"))

f3b1ceb5e1d4d5e8ade18749e4cc4ec7
True


## Question 9.
Prove that nonce is: 0084aa9cfd08200000000008f0


In [15]:
# We extract the needed parts from the packet.
CCMPpar = (multicast_raw_pckg527[24:27] + chr(0x00).encode() + multicast_raw_pckg527[28:32])[::-1]
add_2 = multicast_raw_pckg527[10:16]

# We use the above function to derive it.
multicast_nonce = chr(0x00).encode() + add_2 + CCMPpar[0:2] + CCMPpar[4:8]
print(multicast_nonce.hex())
print(multicast_nonce.hex() == '0084aa9cfd08200000000008f0')

0084aa9cfd08200000000008f0
True


## Question 10.
Compute the ADD of the multicast package. We know that is not a QoS data frame so, we compute it like this: AAD = FrameControl || Address 1 || Address 2 || Address 3 || 0x0000

In [16]:
# We extract the needed parts from the packet.
fc = multicast_raw_pckg527[0:2]
add_1 = multicast_raw_pckg527[4:10]
add_3 = multicast_raw_pckg527[16:22]

# We use the above function.
Multicast_ADD = fc + add_1 + add_2 + add_3 + 2 * chr(0x00).encode()
print(Multicast_ADD.hex())
print(Multicast_ADD.hex() == '0842ffffffffffff84aa9cfd082084aa9cfd081f0000')

0842ffffffffffff84aa9cfd082084aa9cfd081f0000
True


## Question 11.
Decrypt packet 527. To decryp this packet, we follow the previous packet decryption but this time instead of using the TK key, we use the GTK which is the key that protects multicast and broadcast traffic.

In [17]:
data_to_decrypt = multicast_raw_pckg527[32:]
print(data_to_decrypt.hex())
aesccm = AESCCM(GTK, tag_length=8)

multicast_decrypted_data = aesccm.decrypt(multicast_nonce, data_to_decrypt, AAD)

print(multicast_decrypted_data.hex())

bde8115d21d64d8069ac0b3dd70e68d22824136c4e5703294c84a39afcc6e9ee4fa68bd4acfa4f3fbfcedd0e


InvalidTag: 

# 1.5 Dictionary attack
File tradio2.pcapng contains traffic of a wifi network with SSID: Wifi Test but password is unknown. We only now that password consists of the SSID concatenated to a number, having 10 possible passwords.

## Question 12. 
Write a dictionary attack based on checking the integrity of message 2 to derive the correct password. First of all we have to parse the new capture.

In [18]:
capture_file = './tradio2.pcapng'
filter_expr = 'eapol'

attack_handshake_messages = pyshark.FileCapture(capture_file,display_filter=filter_expr, use_json =True, include_raw=True)
packet_1_raw = attack_handshake_messages[0].get_raw_packet()
packet_2_raw = attack_handshake_messages[1].get_raw_packet()

Once we have parsed the capture, according to the statement, the password is the SSID (Wifi_Test) concatenated to a number and knowing that there are 10 possible passwords, we can deduce that this number will go from 0 to 9 (10 possibilitiies). We will apply this ten possibilities, and try to find the password on the handshake messages, trying to compute the MIC with the expected one from the 2n handhshake package.

In [19]:
SSID = 'Wifi_Test'

for i in range(1,10):
    # password to test
    password = f'{SSID}{i}'
    
    # Generate salt based on the SSID
    salt = SSID.encode()
    
    # Define parameters for PBKDF2 key derivation function
    key_length = 32    # Length of the derived key (PMK) in bytes
    iterations = 4096  # Number of iterations

    # Derive PMK using PBKDF2HMAC key derivation function
    kdf = PBKDF2HMAC(
            algorithm=hashes.SHA1(),
            length=key_length,
            salt=salt,
            iterations=iterations,
    )
    pmk = kdf.derive(password.encode())

    MAC_add1 = packet_1_raw[30:36]
    MAC_add2 = packet_1_raw[36:42]
    nonce_1 = packet_1_raw[77:109]
    nonce_2 = packet_2_raw[77:109]

    # ordered the fields (less first)
    mac_1, mac_2 = min(MAC_add1, MAC_add2) , max(MAC_add1, MAC_add2)
    nonce_1, nonce_2 = min(nonce_1, nonce_2), max(nonce_1, nonce_2)         
    
    # generate the data blocks as we did in question 3
    data = 'Pairwise key expansion'.encode() + chr(0x00).encode() + mac_1 + mac_2 + nonce_1 + nonce_2

    # ptk and subkeys
    PTK = SHA1_HMAC(pmk, data, 48)
    KCK = PTK[:16]

    # catch the mic of the second package to compare it with the computed
    expected_mic = packet_2_raw[141:157]
    computed_mic = compute_mic(KCK,packet_2_raw[60:])

    if(expected_mic == computed_mic):
        print('Password found: ' + password)
        break




Password found: Wifi_Test2
